# Modelo de Identificação de Fraudes

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt

from scipy.stats import kstest, normaltest, anderson

from sklearn.preprocessing import PowerTransformer, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, NearMiss 
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.combine import SMOTEENN

from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score,
    precision_recall_curve,
)

import warnings
from src.config import DADOS_MASTERCARD

warnings.filterwarnings('ignore')
sns.set_theme(palette="bright")

In [5]:
# Importando a base

transacoes = pd.read_csv(DADOS_MASTERCARD)

In [4]:
# Verificar se todos os valores podem ser convertidos para inteiro (sem valores decimais)

if (transacoes['Time'] == transacoes['Time'].astype(int)).all():
    
    # Se todos os valores são inteiros, realizar a conversão
    
    transacoes['Time'] = transacoes['Time'].astype(int)
   
else:
    print("Nem todos os valores na coluna 'Time' podem ser convertidos para inteiro.")

### Vamos ajustar a escala de duas colunas para testar o resultado
- As colunas Time e Amount têm valores max muito alto. Destoa do restante do dataset.

In [6]:
# Primeiro para a coluna Time

transacoes.Time = transacoes.Time / transacoes.Time.max()

# E então para a coluna Amount

transacoes.Amount = transacoes.Amount / transacoes.Amount.max()

In [7]:
# Separando X e y

X = transacoes.drop('Class',axis=1)
y = transacoes.Class

**Separando em treino e teste**

In [9]:
# Separando em treino e teste

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0,stratify=y)

In [10]:
# Contando os valores

y_train.value_counts()

Class
0    190490
1       330
Name: count, dtype: int64

### Realizando o random undersampling

In [12]:
# Definindo o RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

In [13]:
# Definindo a nova amostra

X_resRUS, y_resRUS = rus.fit_resample(X_train, y_train)

In [14]:
# Contando os valores após equilibrar o dataset

y_resRUS.value_counts()

Class
0    330
1    330
Name: count, dtype: int64

### Nos passos anteriores, tivemos como os melhores hiperparâmetros:
- **<font color='blue'>Regressão Logística:**
    - {'C': 0.001, 'solver': 'liblinear'}\
- **<font color='blue'>SVC:**
    - {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
- **<font color='blue'>Random Forest:**
    - {'criterion': 'gini', 'max_depth': 9, 'n_estimators': 30}

In [16]:
# Regressão Logística

clf_RL = LogisticRegression(random_state=0,
                             C=0.001,
                             solver='liblinear').fit(X_resRUS, y_resRUS)

y_pred_RL = clf_RL.predict(X_test)

confusion_matrix(y_test,y_pred_RL)

array([[85217,  8608],
       [   10,   152]], dtype=int64)

In [17]:
# SVC

clf_SVC = SVC(random_state=0,
              probability=True,
             C=100,gamma='auto',
             kernel='rbf').fit(X_resRUS, y_resRUS)

y_pred_SVC = clf_SVC.predict(X_test)

confusion_matrix(y_test,y_pred_SVC)

array([[85791,  8034],
       [   12,   150]], dtype=int64)

In [18]:
# Random Forest

clf_RF = RandomForestClassifier(max_depth=9, 
                                random_state=0,
                                criterion='gini',
                                n_estimators=30).fit(X_resRUS, y_resRUS)

y_pred_RF = clf_RF.predict(X_test)

confusion_matrix(y_test,y_pred_RF)

array([[91476,  2349],
       [   19,   143]], dtype=int64)

### Podemos começar adicionando novos parâmetros

**E novamente utilizar o GridSearchCV**

In [21]:
# Adicionando novos parâmetros

parametros = {
    'C': [0.001,0.01,0.1,1,10,100,1000],
    'solver': ['newton-cg','lbfgs','liblinear','sag','saga'],
    'penalty': ['l1', 'l2', 'elasticnet', None],
    'max_iter': [100,500,1000]
}

In [22]:
# Selecionando a Regressão Logística

LogReg = LogisticRegression(random_state=42)

In [24]:
# Criando o classificador

clf_RL2 = GridSearchCV(LogReg, parametros,
                       scoring='recall'
                      ).fit(X_resRUS,y_resRUS)

In [25]:
# E avaliando o modelo

y_pred_RL2 = clf_RL2.predict(X_test)

confusion_matrix(y_test,y_pred_RL2)

array([[78529, 15296],
       [   11,   151]], dtype=int64)

In [26]:
# Verificando o recall com os novos parâmetros

recall_score(y_test, y_pred_RL2)

0.9320987654320988

#### Podemos adicionar novas métricas no scoring (como, por exemplo, adicionar a precisão além do recall)

In [28]:
# Criando o classificador

clf_RL3 = GridSearchCV(LogReg, parametros,
                       scoring=['recall', 'precision'],
                       refit='recall'
                      ).fit(X_resRUS,y_resRUS)

In [29]:
# Avaliando o novo modelo

y_pred_RL3 = clf_RL3.predict(X_test)

confusion_matrix(y_test,y_pred_RL3)

array([[78529, 15296],
       [   11,   151]], dtype=int64)

In [30]:
# Verificando o recall com os novos parâmetros

recall_score(y_test, y_pred_RL3)

0.9320987654320988